In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
import sys
if ('../..' not in sys.path):
    sys.path.append('../..')
print(sys.path)


['/home/kbardool/kusanagi/SparseChemDev/examples/chembl', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python39.zip', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/lib-dynload', '', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages', '/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/IPython/extensions', '/home/kbardool/.ipython', '../..']


In [3]:
import os
import shutil
import subprocess
import sparsechem as sc
import numpy as np
import string
import glob
import scipy.sparse
import pandas as pd
from urllib.request import urlretrieve
from test_train import random_str
import torch 
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import MultiStepLR
from torchinfo import summary
from torch.utils.tensorboard import SummaryWriter
import pprint
import time

pp = pprint.PrettyPrinter(indent=4)

In [4]:
def display_input_parms(args):
    """Display Configuration values."""
    print("\n   Arguments passed :")
    print("   --------------------")
    for a in dir(args):
        if not a.startswith("__") and not callable(getattr(args, a)):
            print("   {:30} {}".format(a, getattr(args, a)))
    print("\n")
    
# def vprint(s=""):
#     if args.verbose:
#         print(s)

 

In [5]:
dev = "cpu"
data_dir="test_chembl23"
rstr = random_str(12)
output_dir = f"./{data_dir}/models-{rstr}/"
print(f"Call test_classification with dev: {dev} , data_dir: {data_dir} \n")  
print(f"Output dir : {output_dir}")

Call test_classification with dev: cpu , data_dir: test_chembl23 

Output dir : ./test_chembl23/models-qrwhyyahcbvi/


## Exploring Variables and Tensors

#### Panda Create Weights Dataframe

Create a dataframe of 100 rows consisting of random training weights and task types. 

In [89]:
    df = pd.DataFrame({
        "task_id":         np.arange(100),
        "training_weight": np.clip(np.random.randn(100), 0, 1),
        "task_type":       np.random.choice(["adme", "panel", "other"], size=100),
    })

In [90]:
df.head(5)

,task_id,training_weight,task_type
0,0,0.600169,adme
1,1,0.000000,adme
2,2,0.616482,panel
3,3,0.335386,panel
4,4,1.000000,adme


In [48]:
conf_file  = glob.glob(f"{output_dir}/*.json")
model_file = glob.glob(f"{output_dir}/*.pt")

print(f"Conf file : {conf_file}")
# results = sc.load_results(conf_file)

Conf file : []


#### Compressed Sparse Row Formatted Matrix Manipulation

In [49]:
ecfp = np.load(f"./{data_dir}/chembl_23mini_x.npy", allow_pickle=True)


In [50]:
print('ecfp     : ' ,type(ecfp))
print('ecfp_item:  ', type(ecfp_item := ecfp.item()),
      '\necfp_tocsr: ', type(ecfp_tocsr := ecfp.item().tocsr()))
if (ecfp_item - ecfp_tocsr).nnz == 0:
    print(' Matricies are equal')

ecfp     :  <class 'numpy.ndarray'>
ecfp_item:   <class 'scipy.sparse.csr.csr_matrix'> 
ecfp_tocsr:  <class 'scipy.sparse.csr.csr_matrix'>
 Matricies are equal


In [51]:
print(repr(ecfp_item))
print(repr(ecfp_tocsr))

<18388x85277 sparse matrix of type '<class 'numpy.float64'>'
	with 1494038 stored elements in Compressed Sparse Row format>
<18388x85277 sparse matrix of type '<class 'numpy.float64'>'
	with 1494038 stored elements in Compressed Sparse Row format>


#### `y_class`

In [52]:
y_class = np.load(f"./{data_dir}/chembl_23mini_y.npy", allow_pickle=True)

print('y_class      : ', type(y_class), y_class.shape)
print('y_class_item : ', type(y_class_item := y_class.item()) , y_class_item.shape,
    '\ny_class_tocsr: ', type(y_class_tocsr := y_class.item().tocsr()), y_class_tocsr.shape)
if (y_class_item - y_class_tocsr).nnz == 0:
    print(' Matricies are equal')
print('Y_class: ', repr(y_class))
print('Y_class_item: ', repr(y_class_item))
print('Y_class_to_csr: ', repr(y_class_tocsr))

y_class      :  <class 'numpy.ndarray'> ()
y_class_item :  <class 'scipy.sparse.csr.csr_matrix'> (18388, 100) 
y_class_tocsr:  <class 'scipy.sparse.csr.csr_matrix'> (18388, 100)
 Matricies are equal
Y_class:  array(<18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>, dtype=object)
Y_class_item:  <18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>
Y_class_to_csr:  <18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>


In [53]:
print((y_class_tocsr == +1).shape)
print((y_class_tocsr > 0.0).shape)

(18388, 100)
(18388, 100)


In [54]:
num_pos    = np.array((y_class_tocsr == +1).sum(0)).flatten()
num_neg    = np.array((y_class_tocsr == -1).sum(0)).flatten()
num_class  = np.array((y_class_tocsr != 0).sum(0)).flatten()
print(f' num pos  : {num_pos[:100]}')
print(f' num neg  : {num_neg[:100]}')
print(f' num class: {num_class[:100]}')

 num pos  : [ 884   59   59  284   55   43  749   14  292  342  423   13  119    1
  281  213    0   11  190   27   46   36   16   43    9   70    7  163
 1399   35  447    1    0  675  327  142    6   31  552   52   27  325
    1  350  535   72   94   36   55  259 1366    3   24    9 1408  148
  971  113   31   55   70   74   63    0   86   58    2    0  465  273
    9  317   91  146  199    0  251  232   62   14  230   79   30   41
   32 1610   74   16   22    0    0    0  102    1    0   32   14    0
   10    1]
 num neg  : [ 83   0   0   0   0   3  29   7   9  11   0   0   0  12   3   0   0   0
   0   9   0  13  21   0   0   0   0   0  45   0   0 143   0   9   0  56
   9  18  17   0   0   1   1   1  72   0  10   0 280   2   0   2  48 245
  17  77   4   0   0   9   0   5   3   2  57   1   0   1   5   2  44   2
  64  11   1  24   3   0 101  21  38   0  19   0   1 220  16  13 400  32
   0  27   1  17   0   0   2   0  32  14]
 num class: [ 967   59   59  284   55   46  778   21  301  3

In [55]:
if (num_class != num_pos + num_neg).any():
    print("For classification all y values (--y_class/--y) must be 1 or -1.")

### `y_censor`

`y_censor` is a sparse matrix : row_count x class_count

In [56]:
y_censor = np.load(f"./{data_dir}/chembl_23mini_y_censored.npy", allow_pickle=True)

print('y_censor      : ', type(y_censor), y_censor.shape)
print('y_censor_item : ', type(y_censor_item := y_censor.item()) , y_censor_item.shape,
    '\ny_censor_tocsr: ', type(y_censor_tocsr := y_censor.item().tocsr()), y_censor_tocsr.shape)
if (y_censor_item - y_censor_tocsr).nnz == 0:
    print(' Matricies are equal')
 
print('Y_censor: ', repr(y_censor))
print('Y_censor_item: ', repr(y_censor_item))
print('Y_censor_to_csr: ', repr(y_censor_tocsr))

y_censor      :  <class 'numpy.ndarray'> ()
y_censor_item :  <class 'scipy.sparse.csr.csr_matrix'> (18388, 100) 
y_censor_tocsr:  <class 'scipy.sparse.csr.csr_matrix'> (18388, 100)
 Matricies are equal
Y_censor:  array(<18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>, dtype=object)
Y_censor_item:  <18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>
Y_censor_to_csr:  <18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>


In [57]:
# print(y_censor_tocsr)


###  `y_regr`

`y_regr` is a sparse matrix : row_count x class_count

In [58]:
y_regr = np.load(f"./{data_dir}/chembl_23mini_y.npy", allow_pickle=True)

print('y_regr      : ', type(y_regr), y_regr.shape)
print('y_regr_item : ', type(y_regr_item := y_regr.item()) , y_regr_item.shape,
    '\ny_regr_tocsr: ', type(y_regr_tocsr := y_regr.item().tocsr()), y_regr_tocsr.shape)
if (y_regr_item - y_regr_tocsr).nnz == 0:
    print(' Matricies y_regr_item and y_regr_tocsr are equal')
 

print('Y_regr: ', repr(y_regr))
print('Y_regr_item: ', repr(y_regr_item))
print('Y_regr_to_csr: ', repr(y_regr_tocsr))

y_regr      :  <class 'numpy.ndarray'> ()
y_regr_item :  <class 'scipy.sparse.csr.csr_matrix'> (18388, 100) 
y_regr_tocsr:  <class 'scipy.sparse.csr.csr_matrix'> (18388, 100)
 Matricies y_regr_item and y_regr_tocsr are equal
Y_regr:  array(<18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>, dtype=object)
Y_regr_item:  <18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>
Y_regr_to_csr:  <18388x100 sparse matrix of type '<class 'numpy.float64'>'
	with 21149 stored elements in Compressed Sparse Row format>


In [59]:
# print(y_regr_tocsr)


### Folding

In [60]:
folding = np.load(f"./{data_dir}/chembl_23mini_folds.npy")
print(repr(folding), folding.shape)
print(np.unique(folding))
print(np.bincount(folding))

array([4, 0, 0, ..., 3, 0, 2]) (18388,)
[0 1 2 3 4]
[3755 3759 3661 3658 3555]


In [61]:
y1= y_class_tocsr[folding == 4]
y1

<3555x100 sparse matrix of type '<class 'numpy.float64'>'
	with 4093 stored elements in Compressed Sparse Row format>

In [62]:
# print(y_class_tocsr)

In [63]:
# print(y1)

In [64]:
# del n_pos, n_neg
n_pos =   np.array((y1 == +1).sum(0))
n_pos.shape
n_pos.flatten()
# n_neg =   np.array((y1 == -1).sum(0)).flatten()

array([182,  27,   7,  60,   6,   5, 157,   3,  44,  91,  62,   6,  17,
         1,  38,  33,   0,   1,  48,  11,  13,   2,   0,   9,   2,  15,
         5,  39, 319,   9, 114,   0,   0, 129,  55,  40,   0,   8, 112,
         6,   0,  31,   0,  62,  80,   7,  17,   0,  10,  67, 254,   1,
         4,   0, 277,  22, 241,   2,  14,  10,   4,  11,   3,   0,  18,
         1,   1,   0,  95,  40,   3,  42,   9,  37,  11,   0,  50,  47,
        22,   0,  47,  15,   0,  10,   6, 326,  12,   5,   7,   0,   0,
         0,  21,   0,   0,   4,   2,   0,   5,   0])

In [65]:
folds = np.unique(folding)
num_pos = []
num_neg = []
for fold in folds:
    yf = y_class_tocsr[folding == fold]
    num_pos.append( np.array((yf == +1).sum(0)).flatten() )
    num_neg.append( np.array((yf == -1).sum(0)).flatten() )
    print(f' Fold: {fold}')
    print(f' ------------')
    print('num_pos: ' , num_pos)
    print('num_neg: ', num_neg)

 Fold: 0
 ------------
num_pos:  [array([115,  31,  17,  54,   7,  18, 203,   3,  55,  76,  63,   1,  42,
         0,  75,  36,   0,   1,  43,   2,   7,   7,   0,   3,   0,  21,
         1,  37, 330,  11,  86,   0,   0, 142,  27,  25,   4,   7, 111,
        12,   7,  79,   0,  97, 129,  15,  10,   9,   9,  38, 272,   1,
         2,   0, 285,  63, 185,   1,   0,   4,  19,  21,  13,   0,  18,
        12,   0,   0,  93,  55,   1,  73,  29,  41,  18,   0,  46,  37,
        14,   0,  40,  18,   6,   8,  10, 330,   6,   4,   1,   0,   0,
         0,   8,   0,   0,   0,   3,   0,   1,   0])]
num_neg:  [array([17,  0,  0,  0,  0,  1,  3,  4,  2,  2,  0,  0,  0,  1,  2,  0,  0,
        0,  0,  2,  0,  0, 13,  0,  0,  0,  0,  0,  6,  0,  0, 30,  0,  0,
        0,  3,  3,  3,  2,  0,  0,  1,  1,  1, 18,  0,  2,  0, 85,  0,  0,
        2, 13, 47,  1, 11,  0,  0,  0,  2,  0,  1,  1,  1,  3,  0,  0,  0,
        0,  0, 18,  0,  9,  3,  0,  4,  1,  0, 23, 11,  6,  0,  5,  0,  0,
       46,  4,  4, 73,

In [66]:
num_pos_stack = np.row_stack(num_pos)

num_pos_stack.shape
num_pos_stack[:,:5]

array([[115,  31,  17,  54,   7],
       [179,   1,  13,  70,   8],
       [237,   0,  10,  63,  15],
       [171,   0,  12,  37,  19],
       [182,  27,   7,  60,   6]])

### Input File Folding and Transformation

In [67]:
folding_size = 2
ecfp_folded = sc.fold_transform_inputs(ecfp_tocsr, folding_size=folding_size)


In [68]:
print(repr(ecfp_tocsr))
print(repr(ecfp_folded))

<18388x85277 sparse matrix of type '<class 'numpy.float64'>'
	with 1494038 stored elements in Compressed Sparse Row format>
<18388x2 sparse matrix of type '<class 'numpy.float64'>'
	with 36776 stored elements in Compressed Sparse Row format>


In [69]:
folding_size = 73
idx = ecfp_tocsr.nonzero()
print('\n', idx[0], idx[0].shape,'\n', idx[1], idx[1].shape)

folded = idx[1] % folding_size
print(folded, len(folded))
x = scipy.sparse.csr_matrix((ecfp_tocsr.data, (idx[0], folded)), shape=(ecfp_tocsr.shape[0], folding_size))
print(repr(x))

## Eliminate dups by adding them together.
x.sum_duplicates()
print(repr(x))


 [    0     0     0 ... 18387 18387 18387] (1494038,) 
 [   10    13    21 ... 84737 84738 84741] (1494038,)
[10 13 21 ... 57 58 61] 1494038
<18388x73 sparse matrix of type '<class 'numpy.float64'>'
	with 899350 stored elements in Compressed Sparse Row format>
<18388x73 sparse matrix of type '<class 'numpy.float64'>'
	with 899350 stored elements in Compressed Sparse Row format>


In [70]:
# print(x)

## Training Setup

In [6]:
from sparsechem.utils import training_arguments, vprint
# %load_ext autoreload
# %autoreload 2
# import sparsechem as sc

ImportError: cannot import name 'vprint' from 'sparsechem.utils' (/home/kbardool/kusanagi/SparseChemDev/examples/chembl/../../sparsechem/utils.py)

In [ ]:
parser = training_arguments()

dev = "cpu" 
data_dir="test_chembl23"
rm_output=False
rstr = random_str(12)
output_dir = f"./{data_dir}/models-{rstr}/"

In [7]:
cmd = (
        f" --x ./{data_dir}/chembl_23mini_x.npy" +
        f" --y_class ./{data_dir}/chembl_23mini_y.npy" +
        f" --folding ./{data_dir}/chembl_23mini_folds.npy" +
        f" --batch_ratio 0.1" +
        f" --output_dir {output_dir}" +
        f" --hidden_sizes 20 30 40 " +
        f" --epochs 2" +
        f" --lr 1e-3" +
        f" --lr_steps 1" +
        f" --dev {dev}" +
        f" --verbose 1"
#         f" --input_size_freq  40"
#         f" --tail_hidden_size  10"
)

args = parser.parse_args(cmd.split())

def vprint(s=""):
    if args.verbose:
        print(s)
# vprint(f"\nArgs : \n--------------")

In [124]:
display_input_parms(args)


   Arguments passed :
   --------------------
   batch_ratio                    0.1
   censored_loss                  1
   dev                            cpu
   epochs                         2
   eval_frequency                 1
   eval_train                     0
   fold_inputs                    None
   fold_te                        None
   fold_va                        0
   folding                        ./test_chembl23/chembl_23mini_folds.npy
   hidden_sizes                   [20, 30, 40]
   input_size_freq                None
   input_transform                none
   internal_batch_max             None
   last_dropout                   0.2
   last_non_linearity             relu
   lr                             0.001
   lr_alpha                       0.3
   lr_steps                       [1]
   middle_dropout                 0.0
   middle_non_linearity           relu
   min_samples_auc                None
   min_samples_class              5
   min_samples_regr               10

In [62]:
##
## Generate runname if one wasn't provided in input args 
##
if args.run_name is not None:
    name = args.run_name
else:
    name  = f"sc_{args.prefix}_h{'.'.join([str(h) for h in args.hidden_sizes])}_ldo{args.last_dropout:.1f}_wd{args.weight_decay}"
    name += f"_lr{args.lr}_lrsteps{'.'.join([str(s) for s in args.lr_steps])}_ep{args.epochs}"
    name += f"_fva{args.fold_va}_fte{args.fold_te}"
print(f"\nRun name is '{name}'.")


Run name is 'sc_run_h20.20_ldo0.2_wd0.0_lr0.001_lrsteps1_ep2_fva0_fteNone'.


#### if args.save_board, Setup tensorboard writer

In [26]:
if args.save_board:
    tb_name = os.path.join(args.output_dir, "boards", name)
    print(f"\nargs.save_board is '{args.save_board}' - will be written to {tb_name}.")
    writer  = SummaryWriter(tb_name)
else:
    writer = Nothing()

assert args.input_size_freq is None, "Using tail compression not yet supported."


args.save_board is '1' - will be written to ./test_chembl23/models-zqwhrybwrbxo/boards/sc_run_h20.20_ldo0.2_wd0.0_lr0.001_lrsteps1_ep2_fva0_fteNone.


AssertionError: Using tail compression not yet supported.

#### load datasets   

In [63]:
## Verify presence of Y label data
if (args.y_class is None) and (args.y_regr is None):
    raise ValueError("No label data specified, please add --y_class and/or --y_regr.")

##



ecfp     = sc.load_sparse(args.x)
y_class  = sc.load_sparse(args.y_class)
y_regr   = sc.load_sparse(args.y_regr)
y_censor = sc.load_sparse(args.y_censor)
 
vprint(f"ecfp shape                   : {ecfp.shape}")
vprint(f"y_class shape                : {y_class.shape}")

if (y_regr is not None):
    vprint(f"y_regr shape         : {y_regr.shape}")

if (y_censor is not None):
    vprint(f"y_censor shape       : {y_censor.shape}")

if (y_regr is None) and (y_censor is not None):
    raise ValueError("y_censor provided please also provide --y_regr.")

if y_class is None:
    y_class = scipy.sparse.csr_matrix((ecfp.shape[0], 0))
    vprint(f"Created y_class shape        : {y_class.shape}")

if y_regr is None:
    y_regr  = scipy.sparse.csr_matrix((ecfp.shape[0], 0))
    vprint(f"Created y_regr shape         : {y_regr.shape}")

if y_censor is None:
    y_censor = scipy.sparse.csr_matrix(y_regr.shape)
    vprint(f"Created y_censor shape       : {y_censor.shape}")
    
folding = np.load(args.folding)
assert ecfp.shape[0] == folding.shape[0], "x and folding must have same number of rows"
vprint(f"folding shape                : {folding.shape}")


vprint() 
vprint(f"x shape              : {ecfp.shape}")
vprint(f"y_class shape        : {y_class.shape}")
vprint(f"y_regr shape         : {y_regr.shape}")
vprint(f"y_censor shape       : {y_censor.shape}")

ecfp shape                   : (18388, 85277)
y_class shape                : (18388, 100)
Created y_regr shape         : (18388, 0)
Created y_censor shape       : (18388, 0)
folding shape                : (18388,)

x shape              : (18388, 85277)
y_class shape        : (18388, 100)
y_regr shape         : (18388, 0)
y_censor shape       : (18388, 0)


#### Loading task weights

In [64]:
type(y_class)

scipy.sparse.csr.csr_matrix

In [65]:
tasks_class = sc.load_task_weights(args.weights_class, y=y_class, label="y_class")
tasks_regr  = sc.load_task_weights(args.weights_regr , y=y_regr, label="y_regr")

vprint(f"tasks_class {tasks_class.training_weight.shape}: \n{tasks_class} \n")
vprint(f"tasks_regr  {tasks_regr.training_weight.shape}: \n{tasks_regr}  \n")

 load_task_weights - filename: None label: y_class
 load_task_weights - no weights file provided, training_weights for all classes set to 1
 load_task_weights - filename: None label: y_regr
 load_task_weights - no weights file provided, training_weights for all classes set to 1
tasks_class torch.Size([100]): 
namespace(aggregation_weight=None, censored_weight=tensor([]), task_type=None, training_weight=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])) 

tasks_regr  torch.Size([0]): 
namespace(aggregation_weight=None, censored_weight=tensor([]), task_type=None, training_weight=tensor([]))  



#### Input folding & transformation

In [66]:
print(f"args.fold_inputs : {args.fold_inputs} \t\t  transform: {args.input_transform}\n")
print(repr(ecfp))
ecfp = sc.fold_transform_inputs(ecfp, folding_size=args.fold_inputs, transform=args.input_transform)##
print(repr(ecfp))

args.fold_inputs : None 		  transform: none

<18388x85277 sparse matrix of type '<class 'numpy.float64'>'
	with 1494038 stored elements in Compressed Sparse Row format>
<18388x85277 sparse matrix of type '<class 'numpy.float64'>'
	with 1494038 stored elements in Compressed Sparse Row format>


#### Get number of positive / neg and total for each classes

In [67]:
num_pos    = np.array((y_class == +1).sum(0)).flatten()
num_neg    = np.array((y_class == -1).sum(0)).flatten()
num_class  = np.array((y_class != 0).sum(0)).flatten()
if (num_class != num_pos + num_neg).any():
    raise ValueError("For classification all y values (--y_class/--y) must be 1 or -1.")

num_regr   = np.bincount(y_regr.indices, minlength=y_regr.shape[1])

print(num_regr)

[]


In [68]:
print(tasks_class.aggregation_weight)
print(folding)

None
[4 0 0 ... 3 0 2]


In [69]:
if tasks_class.aggregation_weight is None:
    '''
    fold classes 
    '''
    ## using min_samples rule
    fold_pos, fold_neg = sc.class_fold_counts(y_class, folding)
    n = args.min_samples_class
    tasks_class.aggregation_weight = ((fold_pos >= n).all(0) & (fold_neg >= n)).all(0).astype(np.float64)

In [70]:
tasks_class.aggregation_weight.shape

(100,)

In [71]:
if tasks_regr.aggregation_weight is None:
    print("proceed")
    if y_censor.nnz == 0:
        y_regr2 = y_regr.copy()
        y_regr2.data[:] = 1
    else:
        ## only counting uncensored data
        y_regr2      = y_censor.copy()
        y_regr2.data = (y_regr2.data == 0).astype(np.int32)
  
    fold_regr, _ = sc.class_fold_counts(y_regr2, folding)
    del y_regr2
    tasks_regr.aggregation_weight = (fold_regr >= args.min_samples_regr).all(0).astype(np.float64)

proceed


In [72]:
vprint(f"Input dimension      : {ecfp.shape[1]}")
vprint(f"#samples             : {ecfp.shape[0]}")
vprint(f"#classification tasks: {y_class.shape[1]}")
vprint(f"#regression tasks    : {y_regr.shape[1]}")
vprint(f"Using {(tasks_class.aggregation_weight > 0).sum()} classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).")
vprint(f"Using {(tasks_regr.aggregation_weight > 0).sum()} regression tasks for calculating metrics (RMSE, Rsquared, correlation).")
vprint(f"args.fold_te         : {args.fold_te}")

Input dimension      : 85277
#samples             : 18388
#classification tasks: 100
#regression tasks    : 0
Using 6 classification tasks for calculating aggregated metrics (AUCROC, F1_max, etc).
Using 0 regression tasks for calculating metrics (RMSE, Rsquared, correlation).
args.fold_te         : None


In [73]:
if args.fold_te is not None and args.fold_te >= 0:
    ## removing test data
    assert args.fold_te != args.fold_va, "fold_va and fold_te must not be equal."
    keep    = folding != args.fold_te
    ecfp    = ecfp[keep]
    y_class = y_class[keep]
    y_regr  = y_regr[keep]
    y_censor= y_censor[keep]
    folding = folding[keep]
else:
    print("args.fold_te is None")

args.fold_te is None


#### Identify training and validation data

Validation data is selected from the folding group which is equal to fold_va.

In [74]:
print(folding.shape)
print(folding[:100])

(18388,)
[4 0 0 3 1 2 4 0 1 0 4 2 4 3 0 0 2 4 0 4 2 1 3 3 3 3 3 3 3 3 3 4 4 4 4 0 0
 1 1 1 1 4 4 4 4 4 4 4 4 0 0 0 0 0 0 0 1 4 4 4 4 4 4 4 4 4 4 0 0 0 0 3 3 3
 3 4 4 4 4 4 4 4 2 2 2 3 3 4 4 2 4 4 4 2 2 2 2 2 2 4]


In [75]:
fold_va = args.fold_va

idx_tr  = np.where(folding != fold_va)[0]
idx_va  = np.where(folding == fold_va)[0]

vprint(f"fold_va        : {fold_va}")
vprint(f"idx_tr         : {idx_tr}    Length: {len(idx_tr)}")
vprint(f"idx_va         : {idx_va}    Length: {len(idx_va)}")

fold_va        : 0
idx_tr         : [    0     3     4 ... 18384 18385 18387]    Length: 14633
idx_va         : [    1     2     7 ... 18377 18380 18386]    Length: 3755


In [76]:
y_class_tr = y_class[idx_tr]
y_class_va = y_class[idx_va]
y_regr_tr  = y_regr[idx_tr]
y_regr_va  = y_regr[idx_va]
y_censor_tr = y_censor[idx_tr]
y_censor_va = y_censor[idx_va]

print(y_class_tr.shape)
print(y_regr_tr.shape)
print(y_censor_tr.shape)
print(y_class_va.shape)
print(y_regr_va.shape)
print(y_censor_va.shape)

(14633, 100)
(14633, 0)
(14633, 0)
(3755, 100)
(3755, 0)
(3755, 0)


In [77]:
num_pos_va  = np.array((y_class_va == +1).sum(0)).flatten()
num_neg_va  = np.array((y_class_va == -1).sum(0)).flatten()
num_regr_va = np.bincount(y_regr_va.indices, minlength=y_regr.shape[1])

print(f' num pos va : {num_pos_va[:100]}  sum: {num_pos_va.sum()}')
print(f' num neg va : {num_neg_va[:100]}  sum: {num_neg_va.sum()}')
print(f' num regr va: {num_regr_va}')

 num pos va : [115  31  17  54   7  18 203   3  55  76  63   1  42   0  75  36   0   1
  43   2   7   7   0   3   0  21   1  37 330  11  86   0   0 142  27  25
   4   7 111  12   7  79   0  97 129  15  10   9   9  38 272   1   2   0
 285  63 185   1   0   4  19  21  13   0  18  12   0   0  93  55   1  73
  29  41  18   0  46  37  14   0  40  18   6   8  10 330   6   4   1   0
   0   0   8   0   0   0   3   0   1   0]  sum: 3804
 num neg va : [17  0  0  0  0  1  3  4  2  2  0  0  0  1  2  0  0  0  0  2  0  0 13  0
  0  0  0  0  6  0  0 30  0  0  0  3  3  3  2  0  0  1  1  1 18  0  2  0
 85  0  0  2 13 47  1 11  0  0  0  2  0  1  1  1  3  0  0  0  0  0 18  0
  9  3  0  4  1  0 23 11  6  0  5  0  0 46  4  4 73  2  0  7  1  0  0  0
  0  0  4  0]  sum: 505
 num regr va: []


#### Determine Batch Size

In [78]:
batch_size  = int(np.ceil(args.batch_ratio * idx_tr.shape[0]))
num_int_batches = 1
print(f" internal_batch_max: {args.internal_batch_max}")
print(f" batch ratio:  {args.batch_ratio}    idx_tr.shape: {idx_tr.shape}")
print(f" batch ratio * idx_tr.shape[0] : {(args.batch_ratio * idx_tr.shape[0])}")
print(f" batch size:  {batch_size} \n\n")

args.internal_batch_max = 1000
if args.internal_batch_max is not None:
    if args.internal_batch_max < batch_size:
       
        print(f" batch_size / args.internal_batch_max : {batch_size / args.internal_batch_max}")
        print(f" num int_batches: {int(np.ceil(batch_size / args.internal_batch_max))}")
        num_int_batches = int(np.ceil(batch_size / args.internal_batch_max)) 
        
        print(f" batch_size / num_int_batches : {batch_size / num_int_batches}")
        print(f" batch_size: {int(np.ceil(batch_size / num_int_batches))}")
        batch_size      = int(np.ceil(batch_size / num_int_batches))
        

vprint(f" internal batch size:   {batch_size}")

 internal_batch_max: None
 batch ratio:  0.1    idx_tr.shape: (14633,)
 batch ratio * idx_tr.shape[0] : 1463.3000000000002
 batch size:  1464 


 batch_size / args.internal_batch_max : 1.464
 num int_batches: 2
 batch_size / num_int_batches : 732.0
 batch_size: 732
 internal batch size:   732


## Instantiate Data Loaders

the sparsechem data loaders return the data in the form of inidices and data.

    x_ind
    x_data
    yc_ind
    yc_data
    
The X indicies and data are used to create a sparse matrix using `torch.sparse_coo_tensor()` :

    X = torch.sparse_coo_tensor(
        batch["x_ind"],
        batch["x_data"],
        size = [batch["batch_size"], loader_tr.dataset.input_size]).to(dev, non_blocking=True) 

In [79]:
ecfp

<18388x85277 sparse matrix of type '<class 'numpy.float64'>'
	with 1494038 stored elements in Compressed Sparse Row format>

In [128]:
dataset_tr = sc.ClassRegrSparseDataset(x=ecfp[idx_tr], y_class=y_class_tr, y_regr=y_regr_tr, y_censor=y_censor_tr)
dataset_va = sc.ClassRegrSparseDataset(x=ecfp[idx_va], y_class=y_class_va, y_regr=y_regr_va, y_censor=y_censor_va)

In [129]:
loader_tr = DataLoader(dataset_tr, batch_size=batch_size, num_workers = 8, pin_memory=True, collate_fn=dataset_tr.collate, shuffle=True)
loader_va = DataLoader(dataset_va, batch_size=batch_size, num_workers = 4, pin_memory=True, collate_fn=dataset_va.collate, shuffle=False)

In [130]:
args.input_size  = dataset_tr.input_size
args.output_size = dataset_tr.output_size
args.input_size_freq = None

args.class_output_size = dataset_tr.class_output_size
args.regr_output_size  = dataset_tr.regr_output_size

In [131]:
print(f" input size        : {dataset_tr.input_size}     output size: {dataset_tr.output_size}")
print(f" class output size : {dataset_tr.class_output_size}     regr_output_size : {dataset_tr.regr_output_size}")

 input size        : 85277     output size: 100
 class output size : 100     regr_output_size : 0


In [132]:
batch_enumerator  = enumerate(loader_tr)

In [133]:
batch_idx, batch = next(batch_enumerator)

print(batch_idx)

0


In [134]:
for i in batch.keys():
    if batch[i] is None:
        continue
    if type(batch[i]) is int:
        print(i , batch[i])
    else:
        print(i,  type(batch[i]), ' sparse tensor: ', batch[i].is_sparse, batch[i].shape)

x_ind <class 'torch.Tensor'>  sparse tensor:  False torch.Size([2, 60175])
x_data <class 'torch.Tensor'>  sparse tensor:  False torch.Size([60175])
yc_ind <class 'torch.Tensor'>  sparse tensor:  False torch.Size([2, 857])
yc_data <class 'torch.Tensor'>  sparse tensor:  False torch.Size([857])
batch_size 732


In [135]:
print(loader_tr.dataset.input_size, loader_tr.dataset.output_size)

85277 100


In [136]:
X = torch.sparse_coo_tensor(
        batch["x_ind"],
        batch["x_data"],
        size = [batch["batch_size"], loader_tr.dataset.input_size]).to(dev, non_blocking=True)        

Y = torch.sparse_coo_tensor(
        batch["yc_ind"],
        batch["yc_data"],
        size = [batch["batch_size"], loader_tr.dataset.output_size]).to(dev, non_blocking=True)    

X.shape, Y.shape

(torch.Size([732, 85277]), torch.Size([732, 100]))

## Model Definition

### Instantiate Model

In [125]:
display_input_parms(args)


   Arguments passed :
   --------------------
   batch_ratio                    0.1
   censored_loss                  1
   dev                            cpu
   epochs                         2
   eval_frequency                 1
   eval_train                     0
   fold_inputs                    None
   fold_te                        None
   fold_va                        0
   folding                        ./test_chembl23/chembl_23mini_folds.npy
   hidden_sizes                   [20, 30, 40]
   input_size_freq                None
   input_transform                none
   internal_batch_max             None
   last_dropout                   0.2
   last_non_linearity             relu
   lr                             0.001
   lr_alpha                       0.3
   lr_steps                       [1]
   middle_dropout                 0.0
   middle_non_linearity           relu
   min_samples_auc                None
   min_samples_class              5
   min_samples_regr               10

In [168]:
%load_ext autoreload
%autoreload 2
del net_old

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [170]:
net_old = sc.SparseFFN(args)
print(net_old)

 defining SparseFFN
----------------------------------------------------------------------------------------------------
 input_size_freq: 85277    input_size: 85277
 self.input_splits: [85277, 0]

 SparseInputNet - init_weights - apply to module SparseLinear(in_features=85277, out_features=20, bias=True)

 SparseInputNet - init_weights - apply to module InputNet(
  (input_net_freq): SparseLinear(in_features=85277, out_features=20, bias=True)
)
----------------------------------------------------------------------------------------------------

 MiddleNet - init_weights - apply to module  ReLU()

 MiddleNet - init_weights - apply to module  Dropout(p=0.0, inplace=False)

 MiddleNet - init_weights - apply to module  Linear(in_features=20, out_features=30, bias=True)

 MiddleNet - init_weights - apply to module  ReLU()

 MiddleNet - init_weights - apply to module  Dropout(p=0.0, inplace=False)

 MiddleNet - init_weights - apply to module  Linear(in_features=30, out_features=40, bias=True

In [191]:
del net

In [192]:
net  = sc.SparseFFN_V2(args)

 defining SparseFFN V2
 input_size_freq: 85277    input_size: 85277
 self.input_splits: [85277, 0]
 SparseNetFFN_V2 - init_weights - apply to module SparseLinear(in_features=85277, out_features=20, bias=True)
 SparseNetFFN_V2 - init_weights - apply to module ReLU()
 SparseNetFFN_V2 - init_weights - apply to module Dropout(p=0.0, inplace=False)
 SparseNetFFN_V2 - init_weights - apply to module Linear(in_features=20, out_features=30, bias=True)
 SparseNetFFN_V2 - init_weights - apply to module ReLU()
 SparseNetFFN_V2 - init_weights - apply to module Dropout(p=0.0, inplace=False)
 SparseNetFFN_V2 - init_weights - apply to module Linear(in_features=30, out_features=40, bias=True)
 SparseNetFFN_V2 - init_weights - apply to module ReLU()
 SparseNetFFN_V2 - init_weights - apply to module Dropout(p=0.2, inplace=False)
 SparseNetFFN_V2 - init_weights - apply to module Linear(in_features=40, out_features=100, bias=True)
 SparseNetFFN_V2 - init_weights - apply to module SparseFFN_V2(
  (InputNet)

In [194]:
print(net)
# print(net.net)

SparseFFN_V2(
  (InputNet): SparseLinear(in_features=85277, out_features=20, bias=True)
  (layer_0_nonlinearity): ReLU()
  (layer_0_dropout): Dropout(p=0.0, inplace=False)
  (layer_0_linear): Linear(in_features=20, out_features=30, bias=True)
  (layer_1_nonlinearity): ReLU()
  (layer_1_dropout): Dropout(p=0.0, inplace=False)
  (layer_1_linear): Linear(in_features=30, out_features=40, bias=True)
  (layer_2_nonlinearity): ReLU()
  (layer_2_dropout): Dropout(p=0.2, inplace=False)
  (layer_2_linear): Linear(in_features=40, out_features=100, bias=True)
)


In [206]:
print(torch.nn.ModuleList(net.modules()))

ModuleList(
  (0): SparseFFN_V2(
    (InputNet): SparseLinear(in_features=85277, out_features=20, bias=True)
    (layer_0_nonlinearity): ReLU()
    (layer_0_dropout): Dropout(p=0.0, inplace=False)
    (layer_0_linear): Linear(in_features=20, out_features=30, bias=True)
    (layer_1_nonlinearity): ReLU()
    (layer_1_dropout): Dropout(p=0.0, inplace=False)
    (layer_1_linear): Linear(in_features=30, out_features=40, bias=True)
    (layer_2_nonlinearity): ReLU()
    (layer_2_dropout): Dropout(p=0.2, inplace=False)
    (layer_2_linear): Linear(in_features=40, out_features=100, bias=True)
  )
  (1): SparseLinear(in_features=85277, out_features=20, bias=True)
  (2): ReLU()
  (3): Dropout(p=0.0, inplace=False)
  (4): Linear(in_features=20, out_features=30, bias=True)
  (5): ReLU()
  (6): Dropout(p=0.0, inplace=False)
  (7): Linear(in_features=30, out_features=40, bias=True)
  (8): ReLU()
  (9): Dropout(p=0.2, inplace=False)
  (10): Linear(in_features=40, out_features=100, bias=True)
)


In [208]:
for i in net.modules():
        print(type(i),  i)
print('----------------------------')        
for i in list(net.children()):
        print(i)
print('----------------------------')             

for i,j in net.named_modules():
    print(f"{i:50s}  {j}")

<class 'sparsechem.models.SparseFFN_V2'> SparseFFN_V2(
  (InputNet): SparseLinear(in_features=85277, out_features=20, bias=True)
  (layer_0_nonlinearity): ReLU()
  (layer_0_dropout): Dropout(p=0.0, inplace=False)
  (layer_0_linear): Linear(in_features=20, out_features=30, bias=True)
  (layer_1_nonlinearity): ReLU()
  (layer_1_dropout): Dropout(p=0.0, inplace=False)
  (layer_1_linear): Linear(in_features=30, out_features=40, bias=True)
  (layer_2_nonlinearity): ReLU()
  (layer_2_dropout): Dropout(p=0.2, inplace=False)
  (layer_2_linear): Linear(in_features=40, out_features=100, bias=True)
)
<class 'sparsechem.models.SparseLinear'> SparseLinear(in_features=85277, out_features=20, bias=True)
<class 'torch.nn.modules.activation.ReLU'> ReLU()
<class 'torch.nn.modules.dropout.Dropout'> Dropout(p=0.0, inplace=False)
<class 'torch.nn.modules.linear.Linear'> Linear(in_features=20, out_features=30, bias=True)
<class 'torch.nn.modules.activation.ReLU'> ReLU()
<class 'torch.nn.modules.dropout.Drop

In [203]:
dev  = torch.device(args.dev)
# net  = sc.SparseFFN_V2(args)
# net  = sc.SparseFFN(args).to(dev)
print(f" dev: {dev}")
print(f" Net: \n {net}")

 dev: cpu
 Net: 
 SparseFFN_V2(
  (InputNet): SparseLinear(in_features=85277, out_features=20, bias=True)
  (layer_0_nonlinearity): ReLU()
  (layer_0_dropout): Dropout(p=0.0, inplace=False)
  (layer_0_linear): Linear(in_features=20, out_features=30, bias=True)
  (layer_1_nonlinearity): ReLU()
  (layer_1_dropout): Dropout(p=0.0, inplace=False)
  (layer_1_linear): Linear(in_features=30, out_features=40, bias=True)
  (layer_2_nonlinearity): ReLU()
  (layer_2_dropout): Dropout(p=0.2, inplace=False)
  (layer_2_linear): Linear(in_features=40, out_features=100, bias=True)
)


    Summarize the given PyTorch model. Summarized information includes:
        1) Layer names,
        2) input/output shapes,
        3) kernel shape,
        4) # of parameters,
        5) # of operations (Mult-Adds)

    Args:
        model (nn.Module):
                PyTorch model to summarize. The model should be fully in either train()
                or eval() mode. If layers are not all in the same mode, running summary
                may have side effects on batchnorm or dropout statistics. If you
                encounter an issue with this, please open a GitHub issue.

    input_data (Sequence of Sizes or Tensors):
            Example input tensor of the model (dtypes inferred from model input).
            - OR -
            Shape of input data as a List/Tuple/torch.Size
            (dtypes must match model input, default is FloatTensors).
            You should NOT include batch size in the tuple.
            - OR -
            If input_data is not provided, no forward pass through the network is
            performed, and the provided model information is limited to layer names.
            Default: None

    batch_dim (int):
            Batch_dimension of input data. If batch_dim is None, the input data
            is assumed to contain the batch dimension.
            WARNING: in a future version, the default will change to None.
            Default: 0

    branching (bool):
            Whether to use the branching layout for the printed output.
            Default: True

    col_names (Iterable[str]):
            Specify which columns to show in the output. Currently supported:
            ("input_size", "output_size", "num_params", "kernel_size", "mult_adds")
            If input_data is not provided, only "num_params" is used.
            Default: ("output_size", "num_params")

    col_width (int):
            Width of each column.
            Default: 25

    depth (int):
            Number of nested layers to traverse (e.g. Sequentials).
            Default: 3

    device (torch.Device):
            Uses this torch device for model and input_data.
            If not specified, uses result of torch.cuda.is_available().
            Default: None

    dtypes (List[torch.dtype]):
            For multiple inputs, specify the size of both inputs, and
            also specify the types of each parameter here.
            Default: None

    verbose (int):
            0 (quiet): No output
            1 (default): Print model summary
            2 (verbose): Show weight and bias layers in full detail
            Default: 1

    *args, **kwargs:
            Other arguments used in `model.forward` function.

    Return:
        ModelStatistics object
                See torchsummary/model_statistics.py for more information.

#### Display Model Summary

In [111]:
# summary(model, input_size=(batch_size, 1, 28, 28))

summary(net, input_size=(200, 85277), 
        col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"],
        verbose=2,
        depth = 6,
        col_width=16,
        row_settings=["depth","var_names"],);

SparseFFN V2 forward - last_hidden : False
Layer (type (var_name):depth-idx)             Input Shape      Output Shape     Param #          Kernel Shape     Mult-Adds
SparseFFN_V2                                  --               --               --               --               --
├─Sequential (net): 1-1                       [200, 85277]     [200, 100]       --               --               --
│    └─0.0.weight                                                               ├─1,705,540      [85277, 20]
│    └─0.0.bias                                                                 ├─20             [20]
│    └─1.layer_0.2.weight                                                       ├─400            [20, 20]
│    └─1.layer_0.2.bias                                                         ├─20             [20]
│    └─2.2.weight                                                               ├─2,000          [100, 20]
│    └─2.2.bias                                                          

In [112]:
summary(net_old, input_size=(200, 85277), 
        col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"],
        verbose=2,
        depth = 6,
        col_width=16,
        row_settings=["depth","var_names"],);

Layer (type (var_name):depth-idx)                  Input Shape      Output Shape     Param #          Kernel Shape     Mult-Adds
SparseFFN                                          --               --               --               --               --
├─Sequential (net): 1-1                            [200, 85277]     [200, 100]       --               --               --
│    └─0.input_net_freq.weight                                                       ├─1,705,540      [85277, 20]
│    └─0.input_net_freq.bias                                                         ├─20             [20]
│    └─1.middle_net.layer_0.2.weight                                                 ├─400            [20, 20]
│    └─1.middle_net.layer_0.2.bias                                                   ├─20             [20]
│    └─2.net.2.weight                                                                ├─2,000          [100, 20]
│    └─2.net.2.bias                                                         

#### Loss functions , Weights, and Optimizer

In [91]:
loss_class = torch.nn.BCEWithLogitsLoss(reduction="none")
loss_regr  = sc.censored_mse_loss

if not args.censored_loss:
    loss_regr = functools.partial(loss_regr, censored_enabled=False)


In [93]:
print(repr(loss_regr))
print(repr(loss_class))

<function censored_mse_loss at 0x7f3c8eecd280>
BCEWithLogitsLoss()


In [94]:
tasks_class.training_weight = tasks_class.training_weight.to(dev)
tasks_regr.training_weight  = tasks_regr.training_weight.to(dev)
tasks_regr.censored_weight  = tasks_regr.censored_weight.to(dev)

optimizer = torch.optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.weight_decay)
scheduler = MultiStepLR(optimizer, milestones=args.lr_steps, gamma=args.lr_alpha)

num_prints = 0

In [100]:
print(optimizer)
print(f"LR steps: {args.lr_steps}   gamma: {args.lr_alpha}")
print(scheduler)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.001
    lr: 0.001
    weight_decay: 0.0
)
LR steps: [1]   gamma: 0.3


In [199]:
from sparsechem import Nothing
# if args.save_board:
#     tb_name = os.path.join(args.output_dir, "boards", name)
#     vprint(f"\nargs.save_board is '{args.save_board}' - will be written to {tb_name}.")
#     writer  = SummaryWriter(tb_name)
# else:
writer = Nothing()

## training loop`

In [207]:
args.epochs = 20
print(args.epochs)


20


In [208]:
##
## Training loop
##
for epoch in range(args.epochs):
    t0 = time.time()
    sc.train_class_regr(
        net, optimizer,
        loader          = loader_tr,
        loss_class      = loss_class,
        loss_regr       = loss_regr,
        dev             = dev,
        weights_class   = tasks_class.training_weight,
        weights_regr    = tasks_regr.training_weight,
        censored_weight = tasks_regr.censored_weight,
        normalize_loss  = args.normalize_loss,
        num_int_batches = num_int_batches,
        progress        = args.verbose >= 2)

    t1 = time.time()

    eval_round = (args.eval_frequency > 0) and ((epoch + 1) % args.eval_frequency == 0)
    last_round = epoch == args.epochs - 1

    if eval_round or last_round:
        results_va = sc.evaluate_class_regr(net, loader_va, 
                                            loss_class, loss_regr, 
                                            tasks_class=tasks_class, tasks_regr=tasks_regr, 
                                            dev=dev, progress = args.verbose >= 2)
        for key, val in results_va["classification_agg"].items():
            writer.add_scalar(key+"/va", val, epoch)
        for key, val in results_va["regression_agg"].items():
            writer.add_scalar(key+"/va", val, epoch)

        if args.eval_train:
            results_tr = sc.evaluate_class_regr(net, loader_tr, 
                                                loss_class, loss_regr, 
                                                tasks_class=tasks_class, tasks_regr=tasks_regr, 
                                                dev=dev, progress = args.verbose >= 2)
            for key, val in results_tr["classification_agg"].items():
                writer.add_scalar(key+"/tr", val, epoch)
            for key, val in results_tr["regression_agg"].items():
                writer.add_scalar(key+"/tr", val, epoch)
        else:
            results_tr = None

        if args.verbose:
            ## printing a new header every 20 lines
            header = num_prints % 20 == 0
            num_prints += 1
            sc.print_metrics_cr(epoch, t1 - t0, results_tr, results_va, header)

    scheduler.step()

writer.close()
vprint()
vprint("Saving performance metrics (AUCs) and model.")

0      |  0.26536  0.34916  0.94332  0.91358  0.92081 |       nan       nan       nan |    0.6 
1      |  0.23169  0.31658  0.95294  0.91591  0.92100 |       nan       nan       nan |    0.6 
2      |  0.20527  0.29097  0.95905  0.91940  0.92214 |       nan       nan       nan |    0.6 
3      |  0.18349  0.26869  0.96191  0.92548  0.92384 |       nan       nan       nan |    0.6 
4      |  0.16677  0.25243  0.96241  0.92780  0.92576 |       nan       nan       nan |    0.6 
5      |  0.15307  0.23865  0.96367  0.93348  0.93068 |       nan       nan       nan |    0.6 
6      |  0.14169  0.22536  0.96466  0.93519  0.93699 |       nan       nan       nan |    0.5 
7      |  0.13215  0.21458  0.96564  0.93764  0.93718 |       nan       nan       nan |    0.6 
8      |  0.12422  0.20382  0.96650  0.93777  0.93718 |       nan       nan       nan |    0.6 
9      |  0.11759  0.19450  0.96663  0.93779  0.93718 |       nan       nan       nan |    0.6 
10     |  0.11179  0.18523  0.96751  0.9

### `train_class_regr()`

In [123]:
# def train_class_regr(net, optimizer, loader, loss_class, loss_regr, dev,
#                      weights_class, weights_regr, censored_weight,
#                      normalize_loss=None, num_int_batches=1, progress=True):
#     sc.train_class_regr(
#         net, 
#         optimizer,
loader          = loader_tr
#         loss_class      = loss_class,
#         loss_regr       = loss_regr,
#         dev             = dev,
weights_class   = tasks_class.training_weight
weights_regr    = tasks_regr.training_weight
censored_weight = tasks_regr.censored_weight
normalize_loss  = args.normalize_loss
num_int_batches = 1
progress        = args.verbose >= 2

In [124]:
net.train()
int_count = 0

In [122]:
# from tqdm  import tqdm
print(type(loader))
print(normalize_loss)
print(num_int_batches)

<class 'torch.utils.data.dataloader.DataLoader'>
None
2


In [105]:
b = next(iter(loader))

In [128]:
print( b.keys())
for key in b.keys():
    if isinstance(b[key], torch.Tensor):
        print(f" {key:10s}:   {b[key].shape}")
    elif isinstance(b[key], int):
        print(f" {key:10s}:   {b[key]}")
    else:
        print(f" {key:10s}:   {b[key]}")

dict_keys(['x_ind', 'x_data', 'yc_ind', 'yc_data', 'ycen_ind', 'ycen_data', 'batch_size'])
 x_ind     :   torch.Size([2, 59843])
 x_data    :   torch.Size([59843])
 yc_ind    :   torch.Size([2, 842])
 yc_data   :   torch.Size([842])
 ycen_ind  :   None
 ycen_data :   None
 batch_size:   732


In [ ]:
i = 0 ;
# for b in tqdm(loader, leave=False, disable=(progress == False)):
# b = next(iter(loader))
print(i, b.keys())
i+=1
for key in b.keys():

    if b[key] is None:
        continue
    if isinstance(b[key], torch.Tensor):
        print(f" {key:10s}:   {b[key].shape}")
    elif isinstance(b[key], int):
        print(f" {key}:   {b[key]}")

if int_count == 0:
    print("optimizer.zero_grad()")
    optimizer.zero_grad()

In [188]:
norm = normalize_loss
print(f"Normalize loss : {norm}")
if norm is None:
    norm = b["batch_size"] * num_int_batches
print(f"Normalize loss : {norm}")

    

Normalize loss : None
Normalize loss : 732


In [180]:
# fwd = batch_forward(net, b=b, 
#                     input_size=loader.dataset.input_size, 
#                     loss_class=loss_class, loss_regr=loss_regr, 
#                     weights_class=weights_class, weights_regr=weights_regr, censored_weight=censored_weight, 
#                     dev=dev)
fwd = out

In [190]:
loss = fwd["yc_loss"] + fwd["yr_loss"]
loss_norm = loss / norm
print(f" loss: {loss}      loss_norm: {loss_norm}    {type(loss_norm)}")


 loss: 555.0751953125      loss_norm: 0.7582994699478149    <class 'torch.Tensor'>


In [191]:
loss_norm.backward()

In [192]:
int_count += 1
print(int_count, num_int_batches)

1 1


In [193]:
if int_count == num_int_batches:
    print(" optimizer.step()")
    optimizer.step()
    int_count = 0

 optimizer.step()


In [194]:
if int_count > 0:
    ## process tail batch (should not happen)
    optimizer.step()

###  `batch_forward()`

In [130]:
# def batch_forward(net, b, input_size, loss_class, loss_regr, weights_class, weights_regr, censored_weight=[], dev="cpu"):


# fwd = batch_forward(net, b=b, 
#                     input_size=loader.dataset.input_size, 
#                     loss_class=loss_class, 
#                     loss_regr=loss_regr, 
#                     weights_class=weights_class, 
#                     weights_regr=weights_regr, 
#                     censored_weight=censored_weight, 
#                     dev=dev)

In [131]:
input_size = loader.dataset.input_size
#                     loss_class = loss_class
#                     loss_regr  = loss_regr
#                     weights_class=weights_class, 
#                     weights_regr=weights_regr, 
#                     censored_weight=censored_weight, 
#                     dev=dev)

In [137]:
print(f" input size : {input_size}")
print(b["x_ind"][:,:4])
print(b["x_ind"][:,-4:])

 input size : 85277
tensor([[ 0,  0,  0,  0],
        [10, 13, 26, 27]])
tensor([[  731,   731,   731,   731],
        [23093, 23094, 23550, 40168]])


In [138]:
X = torch.sparse_coo_tensor(
    b["x_ind"],
    b["x_data"],
    size = [b["batch_size"], input_size]).to(dev, non_blocking=True)

print(X)

tensor(indices=tensor([[    0,     0,     0,  ...,   731,   731,   731],
                       [   10,    13,    26,  ..., 23094, 23550, 40168]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(732, 85277), nnz=59843, layout=torch.sparse_coo)


In [139]:

yc_hat_all, yr_hat_all = net(X)


In [149]:
np.set_printoptions( linewidth=120)
torch.set_printoptions(linewidth=120)

In [150]:
print(yc_hat_all.shape)
print(yc_hat_all[:5,:10])
print(yr_hat_all.shape)

torch.Size([732, 100])
tensor([[ 0.1160, -0.0162,  0.1878,  0.1004, -0.0150,  0.0352,  0.0791,  0.0316,  0.2313,  0.2962],
        [ 0.1471,  0.1062,  0.1394, -0.0030,  0.0516,  0.0937,  0.0319,  0.0841,  0.2850,  0.1720],
        [ 0.2111,  0.0163,  0.1404,  0.0606, -0.0649,  0.1037,  0.0113,  0.0795,  0.2086,  0.2310],
        [ 0.1436,  0.0634,  0.1619,  0.0455, -0.0168,  0.0054,  0.0352,  0.1109,  0.2324,  0.2464],
        [ 0.1452,  0.0182,  0.2110,  0.0465,  0.0157,  0.1497,  0.1226,  0.0303,  0.2124,  0.1476]],
       grad_fn=<SliceBackward>)
torch.Size([732, 0])


In [151]:
out = {}
out["yc_hat_all"] = yc_hat_all
out["yr_hat_all"] = yr_hat_all
out["yc_loss"]    = 0
out["yr_loss"]    = 0
out["yc_weights"] = 0
out["yr_weights"] = 0

In [163]:
print(b["yc_ind"].shape, '\n', b["yc_ind"][:,:20])
print(b["yc_data"].shape, '\n', b["yc_data"][:20])

torch.Size([2, 842]) 
 tensor([[ 0,  1,  2,  3,  3,  4,  5,  6,  7,  8,  9, 10, 11, 11, 12, 13, 14, 15, 16, 17],
        [72, 33,  0, 31, 77, 50, 38, 50, 85,  8, 28, 85, 28, 64, 85, 34, 14,  0,  0, 33]])
torch.Size([842]) 
 tensor([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.])


In [174]:
if net.class_output_size > 0:
    print(net.class_output_size)
    yc_ind  = b["yc_ind"].to(dev, non_blocking=True)
    yc_w    = weights_class[yc_ind[1]]
    yc_data = b["yc_data"].to(dev, non_blocking=True)
    yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
    out["yc_ind"]  = yc_ind
    out["yc_data"] = yc_data
    out["yc_hat"]  = yc_hat
    out["yc_loss"] = (loss_class(yc_hat, yc_data) * yc_w).sum()
    out["yc_weights"] = yc_w.sum()

100


In [177]:
print(b["yc_data"].shape, '\n', b["yc_data"][:20])
print(yc_hat.shape, '\n', yc_hat[:20])
print(yc_w.shape, '\n' , yc_w[:20] )
print(loss_class(yc_hat, yc_data).shape, '\n', loss_class(yc_hat, yc_data)[:20])
print(out["yc_loss"].shape, '\n', out["yc_loss"])

torch.Size([842]) 
 tensor([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.])
torch.Size([842]) 
 tensor([0.1159, 0.1720, 0.2111, 0.1034, 0.0087, 0.1060, 0.1078, 0.1717, 0.0474, 0.2050, 0.2578, 0.0888, 0.2508, 0.2273,
        0.0260, 0.1242, 0.1250, 0.1783, 0.0751, 0.1576], grad_fn=<SliceBackward>)
torch.Size([842]) 
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
torch.Size([842]) 
 tensor([0.6369, 0.6109, 0.5931, 0.7462, 0.6888, 0.6416, 0.6407, 0.6110, 0.6697, 0.5959, 0.5725, 0.6497, 0.5756, 0.8132,
        0.6802, 0.6330, 0.6326, 0.6080, 0.6563, 0.6174], grad_fn=<SliceBackward>)
torch.Size([]) 
 tensor(555.0752, grad_fn=<SumBackward0>)


In [178]:
net.regr_output_size

0

In [179]:
if net.regr_output_size > 0:
    yr_ind  = b["yr_ind"].to(dev, non_blocking=True)
    yr_w    = weights_regr[yr_ind[1]]
    yr_data = b["yr_data"].to(dev, non_blocking=True)
    yr_hat  = yr_hat_all[yr_ind[0], yr_ind[1]]

    out["ycen_data"] = b["ycen_data"]
    if out["ycen_data"] is not None:
        out["ycen_data"] = out["ycen_data"].to(dev, non_blocking=True)

        if len(censored_weight) > 0:
            ## updating weights of censored data
            yrcen_w = yr_w * censored_weight[yr_ind[1]]
            yr_w    = torch.where(out["ycen_data"] == 0, yr_w, yrcen_w)

    out["yr_ind"]  = yr_ind
    out["yr_data"] = yr_data
    out["yr_hat"]  = yr_hat
    out["yr_loss"] = (loss_regr(input=yr_hat, target=yr_data, censor=out["ycen_data"]) * yr_w).sum()
    out["yr_weights"] = yr_w.sum()
else:
    print("net.regr_output_size is 0")

net.regr_output_size is 0


In [59]:

# i = torch.LongTensor((np.arange(10),np.arange(10)))
# v = torch.FloatTensor(np.random.normal(size=10))
# G = torch.sparse.FloatTensor(i,v,torch.Size((100,100)))
# x = torch.Tensor(np.random.normal(size=100))
# device = torch.device('cpu')
# x = x.to(device)
# torch.mm(G, x.unsqueeze(1))

### training functions 

In [209]:

def train_class_regr(net, optimizer, loader, loss_class, loss_regr, dev,
                     weights_class, weights_regr, censored_weight,
                     normalize_loss=None, num_int_batches=1, progress=True):
    
    ## Set the model in training mode.
    net.train()

    int_count = 0
    for b in tqdm(loader, leave=False, disable=(progress == False)):
        if int_count == 0:
            optimizer.zero_grad()

        norm = normalize_loss
        if norm is None:
            norm = b["batch_size"] * num_int_batches

        fwd = batch_forward(net, b=b, 
                            input_size=loader.dataset.input_size, 
                            loss_class=loss_class, loss_regr=loss_regr, 
                            weights_class=weights_class, weights_regr=weights_regr, censored_weight=censored_weight, 
                            dev=dev)
        loss = fwd["yc_loss"] + fwd["yr_loss"]
        loss_norm = loss / norm
    
        loss_norm.backward()

        int_count += 1
        if int_count == num_int_batches:
            optimizer.step()
            int_count = 0

    if int_count > 0:
        ## process tail batch (should not happen)
        optimizer.step()


In [201]:
def batch_forward(net, b, input_size, loss_class, loss_regr, weights_class, weights_regr, censored_weight=[], dev="cpu"):
    """returns full outputs from the network for the batch b"""
    ## Convert CSR tensor to COO tensor.
    X = torch.sparse_coo_tensor(
        b["x_ind"],
        b["x_data"],
        size = [b["batch_size"], input_size]).to(dev, non_blocking=True)

    yc_hat_all, yr_hat_all = net(X)

    out = {}
    out["yc_hat_all"] = yc_hat_all
    out["yr_hat_all"] = yr_hat_all
    out["yc_loss"]    = 0
    out["yr_loss"]    = 0
    out["yc_weights"] = 0
    out["yr_weights"] = 0

    if net.class_output_size > 0:
        yc_ind  = b["yc_ind"].to(dev, non_blocking=True)
        yc_w    = weights_class[yc_ind[1]]
        yc_data = b["yc_data"].to(dev, non_blocking=True)
        yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
        out["yc_ind"]  = yc_ind
        out["yc_data"] = yc_data
        out["yc_hat"]  = yc_hat
        out["yc_loss"] = (loss_class(yc_hat, yc_data) * yc_w).sum()
        out["yc_weights"] = yc_w.sum()

    if net.regr_output_size > 0:
        yr_ind  = b["yr_ind"].to(dev, non_blocking=True)
        yr_w    = weights_regr[yr_ind[1]]
        yr_data = b["yr_data"].to(dev, non_blocking=True)
        yr_hat  = yr_hat_all[yr_ind[0], yr_ind[1]]

        out["ycen_data"] = b["ycen_data"]
        if out["ycen_data"] is not None:
            out["ycen_data"] = out["ycen_data"].to(dev, non_blocking=True)
            
            if len(censored_weight) > 0:
                ## updating weights of censored data
                yrcen_w = yr_w * censored_weight[yr_ind[1]]
                yr_w    = torch.where(out["ycen_data"] == 0, yr_w, yrcen_w)

        out["yr_ind"]  = yr_ind
        out["yr_data"] = yr_data
        out["yr_hat"]  = yr_hat
        out["yr_loss"] = (loss_regr(input=yr_hat, target=yr_data, censor=out["ycen_data"]) * yr_w).sum()
        out["yr_weights"] = yr_w.sum()

    return out

## Model Saving 

In [210]:
#####   model saving   #####
if not os.path.exists(args.output_dir):
   os.makedirs(args.output_dir)

model_file = f"{args.output_dir}/{name}.pt"
out_file   = f"{args.output_dir}/{name}.json"

if args.save_model:
   torch.save(net.state_dict(), model_file)
   vprint(f"Saved model weights into '{model_file}'.")

results_va["classification"]["num_pos"] = num_pos_va
results_va["classification"]["num_neg"] = num_neg_va
results_va["regression"]["num_samples"] = num_regr_va

if results_tr is not None:
    results_tr["classification"]["num_pos"] = num_pos - num_pos_va
    results_tr["classification"]["num_neg"] = num_neg - num_neg_va
    results_tr["regression"]["num_samples"] = num_regr - num_regr_va

sc.save_results(out_file, args, validation=results_va, training=results_tr)

vprint(f"Saved config and results into '{out_file}'.\nYou can load the results by:\n  import sparsechem as sc\n  res = sc.load_results('{out_file}')\n")


NameError: name 'name' is not defined